In [ ]:
# 뉴스기사 크롤링
# 제목 링크 날짜 퍼블리션
# 동적 크롤링, 셀리니움(느리고 방대한데이터에 부적합, 봇으로판단돼서 차단당할수있음_)
# 주의 해야할점
# 1. 너무 많은 요청 보내지않기 
# 2. 사람인것처럼 요청 보내기 User-Agent

In [7]:
import requests
from bs4 import BeautifulSoup
# 2008.01.02&de=2008.01.02 여기부분이 날짜 변경 부분
# 그러면 여기에 날짜를 정해줄수있는 방법이있어야하나?
# 일단 넣어보자
find = "기준금리"
start_date = "2008.01.01"
end_date = "2008.12.31"
page =1 
url = f"https://search.naver.com/search.naver?where=news&query={find}&sm=tab_opt&sort=2&photo=0&field=0&pd=3&ds={start_date}&de={end_date}&docid=&related=0&mynews=1&office_type=3&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom{start_date.replace('.', '')}to{end_date.replace('.', '')}&is_sug_officeid=0&office_category=3&service_area=0&start={page}"

print(url)
res = requests.get(url, headers = {'User-Agent' : "mozilla 5.0"})
res.status_code, res.text.find('큰기쁨예금')

https://search.naver.com/search.naver?where=news&query=기준금리&sm=tab_opt&sort=2&photo=0&field=0&pd=3&ds=2008.01.01&de=2008.12.31&docid=&related=0&mynews=1&office_type=3&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20080101to20081231&is_sug_officeid=0&office_category=3&service_area=0&start=1


(200, 239606)

In [12]:
import requests
from bs4 import BeautifulSoup

find = "기준금리"
# 기존 파일에 이어서 데이터를 추가하고 싶다면 mode='a'
# 날짜를 바꾼다면 mode='a'로 바꿔줘야한다
start_date = "2024.01.01"
end_date = "2024.02.01"
# 아.. 12월 31이 지나면 다시 0부터 시작
# 이제야 알았네...ㅠㅠ

page = 1  
articles = []

while True:
    url = f"https://search.naver.com/search.naver?where=news&query={find}&sm=tab_opt&sort=2&photo=0&field=0&pd=3&ds={start_date}&de={end_date}&docid=&related=0&mynews=1&office_type=3&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom{start_date.replace('.', '')}to{end_date.replace('.', '')}&is_sug_officeid=0&office_category=3&service_area=0&start={page}"
    # f스트링으로 써야 저장한 변수가 적용됨
    # User-Agent 추가 (네이버에서 봇으로 차단되지 않도록)
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")
    # .bx 너무 많은 부분이 떠버림
    news_list = soup.select(".news_area")  
    
    
    # 더 이상 기사가 없으면 종료
    if not news_list:
        break

    for news in news_list:
        title_tag = news.select_one(".news_tit")  # 제목 태그
        press_tag = news.select_one(".info.press")  # 신문사 태그
        date_tag = news.select_one("span.info")  # 날짜 태그 
        
        if title_tag and press_tag and date_tag:
            title = title_tag.text.strip()  # 기사 제목
            link = title_tag["href"]  # 기사 링크
            news_agency = press_tag.text.strip()  # 신문사 이름
            article_date = date_tag.text.strip()  # 날짜
            
            articles.append((title, news_agency, article_date, link))
    
    print(f"{page}번째 페이지 크롤링 완료, 기사 개수: {article_date}")
    # 돌려보니 기사 개수 보단 날짜를 출력해서 남은기간 알수있게 하는게 더좋아보임
    
    # 다음 페이지로 이동
    page += 10  # 10개씩 넘어감 
    # 5개로 줄여봤는데 중복으로 기사를 가져옴

    import time
    import random
    # 차단 방지 1~2초 쉬어 주기
    sleep_time = random.uniform(1,2) # 몰라서 물어봄...
    time.sleep(sleep_time)

for idx,(title, news_agency, article_date, link) in enumerate(articles):
    print(f"'{article_date}','{news_agency}','{title}', '{link}'") 
    # csv 로 저장하려고 , 로 넣었는데 이거 기사제목에도 , 이게 붙지않나?
    # 그러면 각각 문자열로 바꿔주면 기사 제목에 , 도 csv 에선 문자열로 인식
    # 이제 이걸 csv로 저장

import csv
# 기존 파일에 이어서 데이터를 추가하고 싶다면 mode='a'
# 날짜를 바꾼다면 mode='a'로 바꿔줘야한다
with open('article_data1.csv', mode='w',newline='',encoding ='utf-8') as file:
    writer = csv.writer(file)
    # writer.writerow(articles)?
# 이렇게 하니 세로로 된게아니라 한줄로만 써짐
# writerows< s가들어가면 각 요소별로 저장
    writer.writerows(articles)

1번째 페이지 크롤링 완료, 기사 개수: 2024.01.01.
11번째 페이지 크롤링 완료, 기사 개수: 2024.01.01.
21번째 페이지 크롤링 완료, 기사 개수: 2024.01.01.
31번째 페이지 크롤링 완료, 기사 개수: 27면 1단
41번째 페이지 크롤링 완료, 기사 개수: 12면 9단
51번째 페이지 크롤링 완료, 기사 개수: B3면 1단
61번째 페이지 크롤링 완료, 기사 개수: A21면 1단


KeyboardInterrupt: 

In [ ]:
# # csv 로 저장했던 코드
# # CSV 파일을 쓰기 위한 준비
# with open('terminal_data.csv', mode='w', newline='', encoding='utf-8') as file:
#     writer = csv.writer(file)
#     # 헤더 작성
#     writer.writerow(['터미널', '출발', '요금(어른)', '소요시간'])
# news

In [ ]:
# 수정 테스트
import requests
from bs4 import BeautifulSoup
import time # while 문에 포함되어 계속 호출함 위로올려줌
import random
import csv
find = "기준금리"
start_date = "2008.10.02" # 12월 31 넘어가면 2009년으로 넘어가는 조건식이 없음.
end_date = "2008.10.31"
page = 1  
articles = []

while True:
    url = f"https://search.naver.com/search.naver?where=news&query={find}&sm=tab_opt&sort=2&photo=0&field=0&pd=3&ds={start_date}&de={end_date}&docid=&related=0&mynews=1&office_type=3&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom{start_date.replace('.', '')}to{end_date.replace('.', '')}&is_sug_officeid=0&office_category=3&service_area=0&start={page}"

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")
 
    news_list = soup.select(".news_area")  
    

    if not news_list:
        break

    for news in news_list:
        title_tag = news.select_one(".news_tit")  # 제목 태그
        press_tag = news.select_one(".info.press")  # 신문사 태그
        date_tag = news.select_one("span.info")  # 날짜 태그 
        
        if title_tag and press_tag and date_tag:
            title = title_tag.text.strip()  
            link = title_tag["href"]  # 기사 링크
            news_agency = press_tag.text.strip()  
            article_date = date_tag.text.strip()  # 날짜
            
            articles.append((title, news_agency, article_date, link))
    
    print(f"{page}번째 페이지 크롤링 완료, 날짜: {article_date}")
    # 돌려보니 기사 개수 보단 날짜를 출력해서 남은기간 알수있게 하는게 더좋아보임
    
   
    page += len(news_list)  # 이부분이 계속 증가 무한루프 수정
    
    sleep_time = random.uniform(2,5) 
    time.sleep(sleep_time)

for idx,(title, news_agency, article_date, link) in enumerate(articles):
    print(f"'{article_date}','{news_agency}','{title}', '{link}'") 

with open('article_data.csv', mode='a',newline='',encoding ='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(articles)

1번째 페이지 크롤링 완료, 날짜: 2008.10.12.
11번째 페이지 크롤링 완료, 날짜: 2008.10.12.
21번째 페이지 크롤링 완료, 날짜: 2008.10.12.
31번째 페이지 크롤링 완료, 날짜: 2008.10.13.
41번째 페이지 크롤링 완료, 날짜: 2008.10.13.
51번째 페이지 크롤링 완료, 날짜: 2008.10.13.
61번째 페이지 크롤링 완료, 날짜: 2008.10.13.
71번째 페이지 크롤링 완료, 날짜: 2008.10.13.
81번째 페이지 크롤링 완료, 날짜: 2008.10.13.
91번째 페이지 크롤링 완료, 날짜: 2008.10.14.
101번째 페이지 크롤링 완료, 날짜: 2008.10.14.
111번째 페이지 크롤링 완료, 날짜: 2008.10.14.
121번째 페이지 크롤링 완료, 날짜: 2008.10.15.
131번째 페이지 크롤링 완료, 날짜: 2008.10.15.
141번째 페이지 크롤링 완료, 날짜: 2008.10.15.
151번째 페이지 크롤링 완료, 날짜: 2008.10.15.
161번째 페이지 크롤링 완료, 날짜: 2008.10.16.
171번째 페이지 크롤링 완료, 날짜: 2008.10.16.
181번째 페이지 크롤링 완료, 날짜: 2008.10.16.
191번째 페이지 크롤링 완료, 날짜: 2008.10.16.
201번째 페이지 크롤링 완료, 날짜: 2008.10.16.
211번째 페이지 크롤링 완료, 날짜: 2008.10.17.
221번째 페이지 크롤링 완료, 날짜: 2008.10.17.
231번째 페이지 크롤링 완료, 날짜: 2008.10.19.
241번째 페이지 크롤링 완료, 날짜: 2008.10.19.
251번째 페이지 크롤링 완료, 날짜: 2008.10.19.
261번째 페이지 크롤링 완료, 날짜: 2008.10.19.
271번째 페이지 크롤링 완료, 날짜: 2008.10.20.
281번째 페이지 크롤링 완료, 날짜: 2008.10.20.
291번째 페이지 크롤링 완료, 날짜: 200

KeyboardInterrupt: 